In [4]:
# !pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np 
import scipy.stats as spst
import statsmodels.api as sm
import datetime 
import matplotlib.pyplot as plt 
from math import *

In [9]:
# sp500 = yf.Ticker("^GSPC")
# # sp500.info
# hist = sp500.history(period="1mo")
# hist

In [81]:
class Pred_Nextday_Stock: 
    def __init__(self, ticker, train_period = '1y'):
        self.ticker = ticker 
        self.df = None 
        self.training = None
        self.apply = None
        self.period = train_period 
        
    def get_df(self): 
        stock = yf.Ticker(self.ticker)
        self.df = stock.history(period = self.period)
#         return self 
        
    def create_features(self): 
        # Create Return 
        self.df['Ret'] = self.df.Close.pct_change().shift(-1)
        
        # Create 10 days MACD (2 weeks)
        self.df['MACD'] = self.df.Ret.rolling(10).mean().shift(1)
        
        # Create 10 days Bollinger Bands Upper 
        self.df['BBup'] = self.df.Ret.rolling(10).max().shift(1)
        
        # Create monthly std 
        self.df['1MStd'] = self.df.Ret.rolling(20).std(ddof = 1).shift(1)
        
        # Create 5 days direction 
        self.df['Dir'] = (self.df.Close.diff(4).shift(1) >= 0).astype(int)
        
        self.df = sm.add_constant(self.df)
        
        self.training = self.df[['MACD', 'BBup', '1MStd', 'Dir', 'Ret', 'const']].iloc[20:-1]
        self.apply = self.df[['MACD', 'BBup', '1MStd', 'Dir', 'const']].iloc[-1]
        
    def train_model(self): 
        train_X = self.training.drop('Ret', axis = 1)
        train_y = self.training.Ret
        model = sm.OLS(exog = sm.add_constant(train_X), endog = train_y).fit()
#         return model.predict(sm.add_constant(self.apply))
#         self.apply['const'] = 1
#         print(sm.add_constant(train_X))
#         print(model.summary())
#         print(self.apply)
#         print(model.predict(self.apply) )
#         print(model.params)
        return model.predict(self.apply).values[0]

    def model_pred(self):
        self.get_df()
        self.create_features() 
        return self.train_model()

        
#         print(model.summary())
        
S = Pred_Nextday_Stock(ticker = '^GSPC')
# S.get_df()
# S.create_features()
# S.train_model()
# S.training
S.model_pred()

-0.0002795429633603661